In [1]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem

dir_data='/Users/ahenny/'
dir1='/Volumes/LaCie/'
dir2='/Volumes/Extreme Pro/'

In [2]:
ds=xr.open_dataset(dir_data+'holding/'+'merra2_precip_'+str(1981)+'.nc')
precip=ds['precip']

gpcp_threshold_final=xr.zeros_like(precip)
precip.close()
ds.close()

dk=xr.Dataset()
dk['threshold']=(('time','lat','lon'),gpcp_threshold_final.values)
dk.coords['time']=gpcp_threshold_final.time
dk.coords['lat']=gpcp_threshold_final.lat
dk.coords['lon']=gpcp_threshold_final.lon

try:
    os.remove(dir_data+'merra2_threshold_initial_98_all.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'merra2_threshold_initial_98_all.nc',mode='w',format='NETCDF4')

dk.close()
gpcp_threshold_final.close()

/Users/ahenny/anaconda3/lib/python3.9/site-packages/xarray/backends/plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [3]:
####ALL DAYS (INCLUDING DRY)
yrs=np.arange(1991,2021,1)

lon_bins=np.arange(-180,210,30)
lat_bins=np.arange(-90,120,30)
        
dates_all=[]
for i in range(len(yrs)):
    year=yrs[i]
    year_length=365
    if year%4==0:
        year_length=366
            
    date_start=dt.datetime(year,1,1,0)
    date_range=[date_start+dt.timedelta(days=x) for x in range(year_length)]
    dates_all=dates_all+date_range

for j in range(len(lat_bins)-1):
    for k in range(len(lon_bins)-1):
        print((j,k))
        #ds1.close()
        ds1=xr.open_dataset(dir_data+'merra2_threshold_initial_98_all.nc')
        threshold=ds1['threshold'].values
        
        lat_range=[x for x in ds1.lat.values if lat_bins[j]<=x<lat_bins[j+1]]
        lon_range=[x for x in ds1.lon.values if lon_bins[k]<=x<lon_bins[k+1]]
        
        zipped_lat=list(zip(np.arange(ds1.lat.size),ds1.lat.values.tolist()))
        zipped_lon=list(zip(np.arange(ds1.lon.size),ds1.lon.values.tolist()))

        select_lat_indices=[x[0] for x in zipped_lat if x[1] in lat_range]
        select_lon_indices=[x[0] for x in zipped_lon if x[1] in lon_range]
            
        min_lat_index=min(select_lat_indices)
        min_lon_index=min(select_lon_indices)
        max_lat_index=max(select_lat_indices)+1
        max_lon_index=max(select_lon_indices)+1
        
        threshold_test=np.sum(threshold[:,min_lat_index:max_lat_index,min_lon_index:max_lon_index])
        
        if threshold_test==0:
        
            for i in range(len(yrs)):
                year=yrs[i]
                #print(year)

                ds=xr.open_dataset(dir_data+'holding/'+'merra2_precip_'+str(year)+'.nc')
                precip=ds['precip']
                precip=precip.sel(lat=lat_range,lon=lon_range).values

                if i==0:
                    precip_concat=precip
                else:
                    precip_concat=np.concatenate((precip_concat,precip),axis=0)    

            precip_mask=np.ones_like(precip_concat)
            precip_mask=np.where(precip_concat>=0,precip_mask,np.nan)
            precip_mask=np.nansum(precip_mask,axis=0)
            precip_mask=np.where(precip_mask>=0.75*len(precip_concat[:,0,0]),1,np.nan)
            
            dk=xr.Dataset()
            dk['precip']=(('time','lat','lon'),precip_concat)
            dk.coords['time']=dates_all
            dk.coords['lat']=lat_range
            dk.coords['lon']=lon_range

            precip=dk['precip']#.fillna(0)
            
            for i in range(365):
                #print(i)
                ds=xr.open_dataset(dir_data+'threshold_window_'+str(i+1)+'.nc')
                years=ds['years'].values.tolist()
                months=ds['months'].values.tolist()
                days=ds['days'].values.tolist()
                overlap_list=[dt.datetime(x,y,z) for x,y,z in zip(years,months,days)]
                
                overlap_adjust=[x for x in overlap_list if pd.to_datetime(x) in precip.time.values]
                #print(overlap_adjust)
                #sys.exit()

                precip_select=precip.sel(time=overlap_adjust).values#precip within this 61-day window
                threshold_array=np.nanquantile(precip_select,0.98,axis=0)
                threshold_array=np.where(precip_mask==1,threshold_array,np.nan)
                threshold[i,min_lat_index:max_lat_index,min_lon_index:max_lon_index]=threshold_array#.values

            dk=xr.Dataset()
            dk['threshold']=(('time','lat','lon'),threshold)
            dk.coords['time']=ds1.time
            dk.coords['lat']=ds1.lat
            dk.coords['lon']=ds1.lon

            try:
                os.remove(dir_data+'merra2_threshold_initial_98_all.nc')
            except OSError:
                pass
            dk.to_netcdf(dir_data+'merra2_threshold_initial_98_all.nc',mode='w',format='NETCDF4')
            dk.close()
            ds.close()
            ds1.close()

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(0, 10)
(0, 11)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 7)
(1, 8)
(1, 9)
(1, 10)
(1, 11)
(2, 0)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(2, 5)
(2, 6)
(2, 7)
(2, 8)
(2, 9)
(2, 10)
(2, 11)
(3, 0)
(3, 1)
(3, 2)
(3, 3)
(3, 4)
(3, 5)
(3, 6)
(3, 7)
(3, 8)
(3, 9)
(3, 10)
(3, 11)
(4, 0)
(4, 1)
(4, 2)
(4, 3)
(4, 4)
(4, 5)
(4, 6)
(4, 7)
(4, 8)
(4, 9)
(4, 10)
(4, 11)
(5, 0)
(5, 1)
(5, 2)
(5, 3)
(5, 4)
(5, 5)
(5, 6)
(5, 7)
(5, 8)
(5, 9)
(5, 10)
(5, 11)


In [4]:
ds.close()
ds=xr.open_dataset(dir_data+'merra2_threshold_initial_98_all.nc')
threshold=ds['threshold']
print(threshold)

threshold_new=xr.zeros_like(threshold)
for i1 in range(threshold.lat.size):
    print(i1)
    for i2 in range(threshold.lon.size):
        time_series=threshold[:,i1,i2].values.tolist()
        smoothed=[]
        smoothed1=[]
        
        X=20#41-day running mean
        for j in range(len(time_series)):
            if j<X:
                #print(np.arange(len(time_series)-1*X+j,len(time_series),1))
                #print(np.arange(0,j+X+1,1))
                window=time_series[0:j+X+1]+time_series[-1*X+j:]
            elif j>len(time_series)-X-1:
                #print(j)
                #print(np.arange(j-X,len(time_series),1))
                #print(np.arange(0,j+X-len(time_series)+1,1))
                window=time_series[j-X:]+time_series[0:j+X-len(time_series)+1]
                #sys.exit()
            else:
                #print(np.arange(j-X,j+X+1,1))
                window=time_series[j-X:j+X+1]
                #sys.exit()

            window_mean=float(sum(window))/float(len(window))
            smoothed1.append(window_mean)
            
            
        for j in range(len(smoothed1)):
            if j<X:
                #print(np.arange(len(time_series)-1*X+j,len(time_series),1))
                #print(np.arange(0,j+X+1,1))
                window=smoothed1[0:j+X+1]+smoothed1[-1*X+j:]
            elif j>len(smoothed1)-X-1:
                #print(j)
                #print(np.arange(j-X,len(time_series),1))
                #print(np.arange(0,j+X-len(time_series)+1,1))
                window=smoothed1[j-X:]+smoothed1[0:j+X-len(smoothed1)+1]
                #sys.exit()
            else:
                #print(np.arange(j-X,j+X+1,1))
                window=smoothed1[j-X:j+X+1]
                #sys.exit()

            window_mean=float(sum(window))/float(len(window))
            smoothed.append(window_mean)
            
        #sys.exit()
            
        threshold_new[:,i1,i2]=smoothed

<xarray.DataArray 'threshold' (time: 365, lat: 361, lon: 576)>
[75896640 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01 1981-01-02 ... 1981-12-31
  * lat      (lat) float64 -90.0 -89.5 -89.0 -88.5 -88.0 ... 88.5 89.0 89.5 90.0
  * lon      (lon) float64 -180.0 -179.4 -178.8 -178.1 ... 178.1 178.8 179.4
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191

In [5]:
dk=xr.Dataset()
dk['threshold']=(('time','lat','lon'),threshold_new.values)
dk.coords['time']=threshold_new.time
dk.coords['lat']=threshold_new.lat
dk.coords['lon']=threshold_new.lon
try:
    os.remove(dir_data+'merra2_threshold_final_98_all.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'merra2_threshold_final_98_all.nc',mode='w',format='NETCDF4')